<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EB%B6%84%EB%A5%98%EC%9D%98_%ED%8F%89%EA%B0%80_%EB%B0%A9%EB%B2%95%EA%B3%BC_%EC%BB%A8%EC%85%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd


fish = pd.read_csv('/content/Fish.csv')


In [137]:

## 2진 분류를 위해 Perch, Bream 두 어종만 선별
bream_smelt = fish.loc[(fish['Species'] == 'Bream') | (fish['Species'] == 'Perch')]

## 학습데이터와 정답 데이터로 분리
train = bream_smelt[['Length1','Length2', 'Weight','Width']]
target = bream_smelt['Species']

In [138]:
## 학습 데이터와 테스트 데이터로 분리
from sklearn.model_selection import train_test_split
trd, tsd, trt, tst = train_test_split(train, target, train_size=0.8, random_state=42)

In [131]:
## 로지스틱 회귀 모델 학습
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=10000)
lr.fit(trd, trt)
lr.score(trd, trt), lr.score(tsd, tst)

## 회귀 -> 평가 지표(mae, mse, r2)
## 로지스틱 회귀의 score는 어떤 지표? mae, mse, r2 X
## 로지스틱 회귀는 분류. 분류의 평가지표는 회귀와 다르다.

(0.8055555555555556, 0.7368421052631579)

In [127]:
# 분류 평가 지표
## 1. 정확도(accuracy) -> 전체 클래스 중 몇개를 맞췄는가?






In [ ]:
## 정확도의 맹점 -> 양성 클래스를 반영하지 않는다.

trt2 = trt.sort_values()[:32] # trt에서 좀더 자른 버전
# trd2 = trd[trt2.index]
trd2 = trd.loc[trt2.index]

## 무조건 Bream으로 예측하는 함수
def predict_bream(trd) :
  result = []

  for data in trd.values :
      result.append('Bream')

  return result

pred = predict_bream(trd2)

## 예측 결과와 실제 데이터를 비교
matched = 0
for p, t in zip(pred, trt2) :
  print(p, t)

  if p == t :
    matched += 1

matched / len(pred)

## 데이터가 Bream으로 편향되어 있으면 말도 안되는 모델이지만 정확도가 높게 나옴
## 따라서 분류 모델의 경우 정확도만 높게 나오면 될 것이 아니라 다른 보조 지표가 필요함(양성 클래스를 반영한)

In [112]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

## 정확도
print(accuracy_score(trt2, pred))
## 정밀도
print(precision_score(trt2, pred, pos_label='Bream'))
## 재현율
print(recall_score(trt2, pred, pos_label='Bream'))

0.84375
0.84375
1.0


In [ ]:
## TN => [0, 0]
## FP => [0, 1]
## FN => [1, 0]
## TP => [1, 1]
cmat = confusion_matrix(trt2, pred)

TN = cmat[0, 0]
FP = cmat[0, 1]
FN = cmat[1, 0]
TP = cmat[1, 1]

## 정확도
(TN + TP) / (TN + FP + FN + TP)

## 재현율 -> 진짜 양성 중 양성이라고 맞춘 것의 개수(죄다 양성이라고 해버리면 100%가 나오게 됨.)
TP / (TP + FN)

## 정밀도 -> 모델이 양성이라고 한 것 중 실제로 맞는 것의 개수(양성을 음성이라고 한것은 관심 X)
TP / (TP + FP)

## 재현율과 정밀도가 동시에 높으면 베스트
## 하나가 높게 나오고, 하나가 낮게 나오는 경우 -> 낮게 나온 걸 올릴 수 없음
### 정밀도가 중요한 업무가 있고,  ->  실수를 해도 용납 가능하고 최대한 잘 맞추는 게 중요.(스팸 메일, 날씨 예보)
### 재현율이 중요한 업무가 있다.  ->  실수를 용납할 수 없는 업무 (의료, 사기)
## 둘 다 낮게 나오면 모델 성능 안좋음

## 정밀도, 재현율 통합해서 보는 지표 -> F1 - Score (조화평균)


In [139]:
lr.fit(trd, trt)
lr.score(trd, trt)

pred2 = lr.predict(trd)

print(accuracy_score(trt, pred2))
print(precision_score(trt, pred2, pos_label='Bream'))
print(recall_score(trt, pred2, pos_label='Bream'))

0.8055555555555556
0.8421052631578947
0.5925925925925926


In [ ]:
# TN = cmat[0, 0]
# FP = cmat[0, 1]
# FN = cmat[1, 0]
# TP = cmat[1, 1]
confusion_matrix(trt, pred2)

In [143]:
## 다중 분류 -> 여러 클래스중 특정 클래스일 확률을 예측
mtrain = fish.drop('Species', axis=1)
mtarget = fish['Species']

# stratify => 데이터를 분리할 때 원본 샘플의 비율을 최대한 유지
trd, tsd, trt, tst = train_test_split(mtrain, mtarget, stratify=mtarget,train_size=0.8, random_state=42)


In [146]:
lr3 = LogisticRegression(max_iter=10000)

lr3.fit(trd, trt)
lr3.score(trd, trt), lr3.score(tsd, tst)

(0.9763779527559056, 0.96875)

In [ ]:
import numpy as np

  # 과학적 표기법 대신 소수점 6자리까지 나타낸다.
np.set_printoptions(precision=6, suppress=True)
pd.options.display.float_format = '{:.5f}'.format

pred3 = lr3.predict(tsd)

## 각 클래스에대한 확률
print(lr3.predict_proba(tsd[:10]))

## 클래스 확인
print(lr3.classes_)
print(tst[:10])




In [ ]:
## 피처 중요도
### 행의 의미 -> 클래스 개수
### 열의 의미 -> 특성의 개수

### 모델이 사용한 특성 목록
lr3.feature_names_in_

### 모델이 구별한 클래스(범주) 목록
lr3.classes_

pd.DataFrame(lr3.coef_, columns=lr3.feature_names_in_, index=lr3.classes_) # 각 클래스별 중요 특성 확인 가능

In [ ]:
## 정밀도, 재현율

# precision_score(pred3, tst)
## classification_report

## 다중 분류의 경우 classification_report를 이용해 각 클래스에 대한 정밀도와 재현율을 볼 수 있음.
from sklearn.metrics import classification_report
print(classification_report(pred3, tst))
